# PRMBlock Tutorial

In [8]:
# Standard imports
from datasets import Dataset

# Import our PRM Block
from sdg_hub.blocks.rmblocks import PRMBlock

## Install sdg-hub


```bash 
pip install sdg-hub==0.1.0a2
```

## 1. vLLM serve

Let's serve the qwen math process reward model

```bash
vllm serve Qwen/Qwen2.5-Math-PRM-7B --task reward --tensor-parallel-size 1
```

NOTE: Remember to serve with task set as reward

## 2. Reward Task

Let's create a simple dataset with one math problem and a step by step response

In [4]:
# Create a sample dataset with one math problem
question = "Sue lives in a fun neighborhood.  One weekend, the neighbors decided to play a prank on Sue.  On Friday morning, the neighbors placed 18 pink plastic flamingos out on Sue's front yard.  On Saturday morning, the neighbors took back one third of the flamingos, painted them white, and put these newly painted white flamingos back out on Sue's front yard.  Then, on Sunday morning, they added another 18 pink plastic flamingos to the collection. At noon on Sunday, how many more pink plastic flamingos were out than white plastic flamingos?"
response = "To find out how many more pink plastic flamingos were out than white plastic flamingos at noon on Sunday, we can break down the problem into steps. First, on Friday, the neighbors start with 18 pink plastic flamingos.\n\nOn Saturday, they take back one third of the flamingos. Since there were 18 flamingos, (1/3 \\times 18 = 6) flamingos are taken back. So, they have (18 - 6 = 12) flamingos left in their possession. Then, they paint these 6 flamingos white and put them back out on Sue's front yard. Now, Sue has the original 12 pink flamingos plus the 6 new white ones. Thus, by the end of Saturday, Sue has (12 + 6 = 18) pink flamingos and 6 white flamingos.\n\nOn Sunday, the neighbors add another 18 pink plastic flamingos to Sue's front yard. By the end of Sunday morning, Sue has (18 + 18 = 36) pink flamingos and still 6 white flamingos.\n\nTo find the difference, subtract the number of white flamingos from the number of pink flamingos: (36 - 6 = 30). Therefore, at noon on Sunday, there were 30 more pink plastic flamingos out than white plastic flamingos. The answer is (\\boxed{30})."

data = {
    "question": question,
    "response": response
}

dataset = Dataset.from_list([data])

## 3. Setup PRM Block

In [5]:
# Initialize the PRM Block
prmblock = PRMBlock(
    block_name="math_reward",
    host="0.0.0.0",
    port=8000,
    model_name="Qwen/Qwen2.5-Math-PRM-7B",
    prompt_col="question",
    response_col="response",
    system_prompt="Please reason step by step, and put your final answer within \\boxed{}.",
    step_separator="\n\n",
    step_fill_token="<extra_0>" # this is a model specific fill token
)

[15:54:31] INFO     Initialized PRMBlock with API URL: http://0.0.0.0:8000/pooling                   ]8;id=454552;file:///workspace/home/lab/shiv/SDG-Research/src/sdg_hub/blocks/rmblocks.py\rmblocks.py]8;;\:]8;id=631084;file:///workspace/home/lab/shiv/SDG-Research/src/sdg_hub/blocks/rmblocks.py#56\56]8;;\

## 3. Generate Rewards

In [11]:
# Get rewards
result = prmblock.generate(dataset)

print(f"Question: {result[0]['question']}\n")

for i, (step, reward) in enumerate(zip(result[0]["response"].split("\n\n"), result[0]["step_rewards"]), 1):
    print(f"Step {i}: {step}")
    print(f"Reward: {reward:.3f}\n")

Map: 100%|██████████| 1/1 [00:00<00:00, 41.72 examples/s]

Question: Sue lives in a fun neighborhood.  One weekend, the neighbors decided to play a prank on Sue.  On Friday morning, the neighbors placed 18 pink plastic flamingos out on Sue's front yard.  On Saturday morning, the neighbors took back one third of the flamingos, painted them white, and put these newly painted white flamingos back out on Sue's front yard.  Then, on Sunday morning, they added another 18 pink plastic flamingos to the collection. At noon on Sunday, how many more pink plastic flamingos were out than white plastic flamingos?

Step 1: To find out how many more pink plastic flamingos were out than white plastic flamingos at noon on Sunday, we can break down the problem into steps. First, on Friday, the neighbors start with 18 pink plastic flamingos.
Reward: 1.000

Step 2: On Saturday, they take back one third of the flamingos. Since there were 18 flamingos, (1/3 \times 18 = 6) flamingos are taken back. So, they have (18 - 6 = 12) flamingos left in their possession. Then,

Note: pass the `batch_size` arg which accepts an int to the generate method for faster generations